

```
Key Adaptation of combined script:
Degree Days Calculation: The aggregate_monthly_data function now includes a loop over the range of base temperatures from -20°C to 35°C, creating a separate band for each degree day calculation. These are combined into a single image for each month and year.

Band Naming: Each degree day band is named according to the base temperature, ensuring they are distinct and correctly labeled.

Aggregation and Export: The aggregated data is processed and exported as a CSV, with each degree day band included in the output.
```



In [ ]:
PROJECT = "era5-land-project"

In [ ]:
!gcloud auth login --project {PROJECT}



You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

In [ ]:
import datetime
import ee
import logging # to check task status when exporting images
import math


In [ ]:
ee.Authenticate()
ee.Initialize(project=PROJECT)

# OLD: Calculate Monthly Max Temperature, Total Precipitation, and Degree Days
monthly_aggregation.py

In [ ]:


# # Define functions for aggregation
# def monthly_max_temp(year, month):
#     start_date = ee.Date.fromYMD(year, month, 1)
#     end_date = start_date.advance(1, 'month')
#     era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
#     monthly_data = era_hourly.filterDate(start_date, end_date).select('temperature_2m')
#     monthly_max = monthly_data.max().set('system:time_start', start_date.millis())
#     return monthly_max

# def monthly_total_precip(year, month):
#     start_date = ee.Date.fromYMD(year, month, 1)
#     end_date = start_date.advance(1, 'month')
#     era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
#     monthly_data = era_hourly.filterDate(start_date, end_date).select('total_precipitation')
#     monthly_total = monthly_data.sum().set('system:time_start', start_date.millis())
#     return monthly_total

# def degree_days(year, month, base_temp):
#     start_date = ee.Date.fromYMD(year, month, 1)
#     end_date = start_date.advance(1, 'month')
#     era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
#     monthly_data = era_hourly.filterDate(start_date, end_date).select('temperature_2m')
#     degree_days = monthly_data.map(lambda img: img.subtract(base_temp).max(0)).sum().set('system:time_start', start_date.millis())
#     return degree_days

# def generate_monthly_aggregates(years, base_temp=0):
#     monthly_aggregates = []
#     for year in years:
#         for month in range(1, 13):
#             monthly_max = monthly_max_temp(year, month)
#             monthly_precip = monthly_total_precip(year, month)
#             degree_day = degree_days(year, month, base_temp)
#             combined = monthly_max.addBands(monthly_precip.rename('total_precip')).addBands(degree_day.rename('degree_days_above_0'))
#             combined = combined.set('year', year).set('month', month)
#             monthly_aggregates.append(combined)
#     return ee.ImageCollection(monthly_aggregates)

# # Define the years for aggregation
# years = [2022, 2023]

# # Generate monthly aggregates
# monthly_aggregates_ic = generate_monthly_aggregates(years)


# Updated: Generic Aggregate function for different types of temperature stats

In [ ]:
def aggregate_monthly_data(year, month, band_name, operation, base_temps=None):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
    monthly_data = era_hourly.filterDate(start_date, end_date).select(band_name)


    if operation == 'degree_days' and base_temps is not None:
        degree_days_images = []
        for base_temp in base_temps:
            degree_days = monthly_data.map(lambda img: img.subtract(base_temp).max(0)).sum()
            degree_days = degree_days.rename(f'degree_days_above_{base_temp}')
            degree_days_images.append(degree_days)
        aggregated_data = ee.Image.cat(degree_days_images)
    elif operation == 'max':
        aggregated_data = monthly_data.max()
    else:
        raise ValueError(f"Unsupported operation: {operation}")

    # Convert temperature from Kelvin to Celsius if the band name includes 'temperature'
    if 'temperature' in band_name:
        aggregated_data = aggregated_data.subtract(273.15)

    return aggregated_data.set('system:time_start', start_date.millis())


In [ ]:
def generate_monthly_aggregates(years, base_temps):
    monthly_aggregates = []
    for year in years:
        for month in range(1, 13):
            monthly_max = aggregate_monthly_data(year, month, 'temperature_2m', 'max')
            degree_days = aggregate_monthly_data(year, month, 'temperature_2m', 'degree_days', base_temps)

            year_band = ee.Image.constant(year).rename('year').toFloat()
            month_band = ee.Image.constant(month).rename('month').toFloat()

            combined = monthly_max.rename('max_temp') \
                                  .addBands(degree_days) \
                                  .addBands(year_band) \
                                  .addBands(month_band)

            monthly_aggregates.append(combined)
    return ee.ImageCollection(monthly_aggregates)


In [ ]:
base_temps = list(range(-20, 36))
monthly_aggregates_ic = generate_monthly_aggregates([2023], base_temps)
print(monthly_aggregates_ic.size().getInfo())
print(monthly_aggregates_ic.first().bandNames().getInfo())
print(monthly_aggregates_ic.first().getInfo())

12
['max_temp', 'degree_days_above_-20', 'degree_days_above_-19', 'degree_days_above_-18', 'degree_days_above_-17', 'degree_days_above_-16', 'degree_days_above_-15', 'degree_days_above_-14', 'degree_days_above_-13', 'degree_days_above_-12', 'degree_days_above_-11', 'degree_days_above_-10', 'degree_days_above_-9', 'degree_days_above_-8', 'degree_days_above_-7', 'degree_days_above_-6', 'degree_days_above_-5', 'degree_days_above_-4', 'degree_days_above_-3', 'degree_days_above_-2', 'degree_days_above_-1', 'degree_days_above_0', 'degree_days_above_1', 'degree_days_above_2', 'degree_days_above_3', 'degree_days_above_4', 'degree_days_above_5', 'degree_days_above_6', 'degree_days_above_7', 'degree_days_above_8', 'degree_days_above_9', 'degree_days_above_10', 'degree_days_above_11', 'degree_days_above_12', 'degree_days_above_13', 'degree_days_above_14', 'degree_days_above_15', 'degree_days_above_16', 'degree_days_above_17', 'degree_days_above_18', 'degree_days_above_19', 'degree_days_above_20',

Main Changes by July 27, 2024

- ***Year and Month as Bands***: The year and month are added as bands to each image using ee.Image.constant(year).rename('year') and ee.Image.constant(month).rename('month'). This ensures that they are included in the exported data.
- ***Metadata Handling for Export***: The export function now uses reduceRegion with ee.Reducer.first() to extract the year and month from the image bands instead of using metadata. This approach ensures that these values are correctly extracted from the bands added to the image.
- ***Error Handling***: The function skips the export if year or month is missing, with a clear message indicating why.






# Exporitng aggregates to GEE as ImageCollection Asset

In [ ]:
# # Function to export each image in the collection to an asset
# def export_to_asset(image, index, asset_id_prefix):
#     image_id = ee.Number(index).format().getInfo()
#     asset_id = f"{asset_id_prefix}_{image_id}"
#     description = f"Export_monthly_aggregates_{image_id}"
#     task = ee.batch.Export.image.toAsset(
#         image=image,
#         description=description,
#         assetId=asset_id,
#         scale=1000,
#         region=ee.Geometry.Rectangle([-180, -90, 180, 90])
#     )
#     task.start()
#     print(f"Exporting {description} to {asset_id}")

# # Define the asset ID prefix
# asset_id_prefix = f'projects/{PROJECT}/assets/monthly_aggregates_22_23'

# # Export each image in the collection
# image_list = monthly_aggregates_ic.toList(monthly_aggregates_ic.size())
# for i in range(image_list.size().getInfo()):
#     export_to_asset(ee.Image(image_list.get(i)), i, asset_id_prefix)


# Exporting monthly aggregates to Google Drive in GEOTIFF format

Key Fix:
- Point Geometry for reduceRegion: The reduceRegion function now includes a point parameter to define a specific location for reduction. This resolves the issue of working with unbounded images by specifying a point on the globe.

In [ ]:
# # Function to export each image in the collection to Google Drive
# def export_to_drive(image, description_prefix, folder):
#     # Define a point or a small region to reduce the image over
#     point = ee.Geometry.Point([0, 0])  # Arbitrarily chosen point
#     year = image.select('year').reduceRegion(ee.Reducer.first(), point, 1000).get('year').getInfo()
#     month = image.select('month').reduceRegion(ee.Reducer.first(), point, 1000).get('month').getInfo()
#     if year is None or month is None:
#         print(f"Skipping export for image due to missing year or month bands.")
#         return
#     description = f"{description_prefix}_{int(year)}_{int(month):02d}"
#     export_params = {
#         'image': image,
#         'description': description,
#         'folder': folder,
#         'scale': 1000,
#         'region': ee.Geometry.Rectangle([-180, -90, 180, 90]),
#         'fileFormat': 'GeoTIFF'
#     }
#     task = ee.batch.Export.image.toDrive(**export_params)
#     task.start()
#     print(f"Exporting {description} to Google Drive")

# # Define the years for aggregation
# years = [2022, 2023]

# # Generate monthly aggregates
# monthly_aggregates_ic = generate_monthly_aggregates(years)

# # Define the export parameters
# description_prefix = 'monthly_aggregates'
# folder = 'export_era5_monthly_2_years_27jul'

# # Export each image in the collection
# image_list = monthly_aggregates_ic.toList(monthly_aggregates_ic.size())
# for i in range(image_list.size().getInfo()):
#     export_to_drive(ee.Image(image_list.get(i)), description_prefix, folder)

In [ ]:
# # Function to export each image in the collection to Google Drive
# def export_to_drive(image, description_prefix, folder):
#     year = image.get('year').getInfo()
#     month = image.get('month').getInfo()
#     if year is None or month is None:
#         print(f"Skipping export for image due to missing year or month metadata.")
#         return
#     description = f"{description_prefix}_{year}_{int(month):02d}"
#     export_params = {
#         'image': image,
#         'description': description,
#         'folder': folder,
#         'scale': 1000,
#         'region': ee.Geometry.Rectangle([-180, -90, 180, 90]),
#         'fileFormat': 'GeoTIFF'
#     }
#     task = ee.batch.Export.image.toDrive(**export_params)
#     task.start()
#     print(f"Exporting {description} to Google Drive")

# # Define the export parameters
# description_prefix = 'monthly_aggregates'
# folder = 'export_era5_monthly_2years_aggregates'
# #
# # Export each image in the collection
# image_list = monthly_aggregates_ic.toList(monthly_aggregates_ic.size())
# for i in range(image_list.size().getInfo()):
#     export_to_drive(ee.Image(image_list.get(i)), description_prefix, folder)




> Link to output:
> https://drive.google.com/drive/folders/1Hr1q8DE10YvcwEFdUzyzIMLEBilLbbsa?usp=sharing



# Exporting in CSV

- used zonal_stats function for computing statistics over an image collection based on a feature collection.
- Coordinates FeatureCollection: Replaced this with actual feature collection.

- Export Parameters: The Export.table.toDrive function exports the resulting feature collection to Google Drive in CSV format.

In [ ]:
coordinates = ee.FeatureCollection("projects/era5-land-project/assets/itrdb_locations_unique_with_duplicate_lat_lon_info")


In [ ]:
def zonal_stats(ic, fc, params=None):
    _params = {
        'reducer': ee.Reducer.mean(),
        'scale': None,
        'crs': None,
        'bands': None,
        'bandsRename': None,
        'imgProps': None,
        'imgPropsRename': None,
        'datetimeName': 'datetime',
        'datetimeFormat': 'YYYY-MM-dd HH:mm:ss'
    }
    if params:
        for param in params:
            _params[param] = params[param] or _params[param]

    img_rep = ic.first()
    non_system_img_props = ee.Feature(None).copyProperties(img_rep).propertyNames()
    if not _params['bands']:
        _params['bands'] = img_rep.bandNames()
    if not _params['bandsRename']:
        _params['bandsRename'] = _params['bands']
    if not _params['imgProps']:
        _params['imgProps'] = non_system_img_props
    if not _params['imgPropsRename']:
        _params['imgPropsRename'] = _params['imgProps']

    def map_function(img):
        img = ee.Image(img.select(_params['bands'], _params['bandsRename']))
        img = img.set(_params['datetimeName'], img.date().format(_params['datetimeFormat']))
        img = img.set('timestamp', img.get('system:time_start'))
        props_from = ee.List(_params['imgProps']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        props_to = ee.List(_params['imgPropsRename']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        img_props = img.toDictionary(props_from).rename(props_from, props_to)
        fc_sub = fc.filterBounds(img.geometry())
        return img.reduceRegions(
            collection=fc_sub,
            reducer=_params['reducer'],
            scale=_params['scale'],
            crs=_params['crs']
        ).map(lambda f: f.set(img_props))

    results = ic.map(map_function).flatten()
    return results


In [ ]:
# Apply zonal statistics to the monthly aggregates
params = {'reducer': ee.Reducer.mean(), 'scale': 1000}
ptsERA5monthly = zonal_stats(monthly_aggregates_ic, coordinates, params)

# Export to Google Drive
export_params = {
    'collection': ptsERA5monthly,
    'folder': 'export_era5_monthly_new',
    'description': "monthly_aggregates_export_2023_all_degree_days",
    'fileFormat': 'CSV'
}

task = ee.batch.Export.table.toDrive(**export_params)
task.start()
print("Exporting monthly aggregates to Google Drive")

Exporting monthly aggregates to Google Drive


#### Link to Output:
https://drive.google.com/drive/folders/1KXZrEP5CPJFNo1z09smH03CDWKFQAEh0?usp=sharing

- complete